In [1]:
import whisper

model = whisper.load_model("base") #"large-v3")

# load audio and pad/trim it to fit 30 seconds
# audio = whisper.load_audio("audio.mp3")
# audio = whisper.pad_or_trim(audio)

# # make log-Mel spectrogram and move to the same device as the model
# mel = whisper.log_mel_spectrogram(audio).to(model.device)

# # detect the spoken language
# _, probs = model.detect_language(mel)
# print(f"Detected language: {max(probs, key=probs.get)}")

# # decode the audio
# options = whisper.DecodingOptions()
# result = whisper.decode(model, mel, options)

# # print the recognized text
# print(result.text)

In [3]:
model


Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1280, out_features=1280, bias=True)
          (key): Linear(in_features=1280, out_features=1280, bias=False)
          (value): Linear(in_features=1280, out_features=1280, bias=True)
          (out): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=1280, out_features=5120, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (mlp_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm(

In [2]:
audio = whisper.load_audio("/home/gs/Documents/zhaozhongxiang.wav")#eng.mp3")#
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio, n_mels = 80).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: zh
都是一个动群和一个入侵证


In [3]:
emb = model.embed_audio(mel[None,...])
# mel[None,...].shape

In [5]:
mel.shape,emb.shape

(torch.Size([80, 3000]), torch.Size([1, 1500, 512]))

In [1]:
from datasets import load_dataset, DatasetDict

common_voice1 = DatasetDict()

common_voice1["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="train", use_auth_token=False).select(range(1000))
# common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="train+validation", use_auth_token=True)
common_voice1["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="test", use_auth_token=False).select(range(50))

print(common_voice1)


/home/gs/miniconda3/envs/t2v/lib/python3.10/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/home/gs/miniconda3/envs/t2v/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/gs/miniconda3/envs/t2v/lib/python3.10/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 50
    })
})


In [111]:
common_voice = common_voice1.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
print(common_voice)
# tokenizer(["hello"]).input_ids
# common_voice.column_names["train"]

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 50
    })
})


In [112]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor
from datasets import Audio

model_default = "openai/whisper-small"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_default, language="zh")
tokenizer = WhisperTokenizer.from_pretrained(model_default, task="transcribe", language="zh")
processor = WhisperProcessor.from_pretrained(model_default, task="transcribe", language="zh")




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [114]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    # print(f"""** {batch}""")
    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    # print(f"""** {batch["labels"]}""")
    return batch

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [115]:
common_voice

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 50
    })
})

In [116]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

metric = evaluate.load("wer")


In [117]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.generation_config.language = "zh"

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


In [77]:
# from transformers import WhisperConfig, WhisperModel
# configuration = WhisperConfig.from_pretrained("openai/whisper-small", language="zh")
# model = WhisperModel(configuration)
# model.config.forced_decoder_ids = None
# model.config.suppress_tokens = []
feature_extractor

WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [118]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-zh-me",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100, #500,
    max_steps=1000, #4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500, #500, #1000,
    eval_steps=500, #500, #1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False, #True,


)
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
        # language="zh",
)
trainer.train()


/home/gs/miniconda3/envs/t2v/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,0.027500,1.413643,82.000000
1000,0.002100,1.454970,78.000000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}
/home/gs/miniconda3/envs/t2v/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https:/

TrainOutput(global_step=1000, training_loss=0.27762053986638785, metrics={'train_runtime': 1451.7643, 'train_samples_per_second': 11.021, 'train_steps_per_second': 0.689, 'total_flos': 4.5827361570816e+18, 'train_loss': 0.27762053986638785, 'epoch': 15.87})

In [4]:
#evaluation

from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load
from datasets import Audio

# librispeech_test_clean = load_dataset("librispeech_asr", "clean", split="test")

model_pth = "/home/gs/work/audiolm-pytorch/whisper/whisper-small-zh-me/checkpoint-500" #(
model_token_pth = "openai/whisper-small"
# model_pth = "openai/whisper-large"
is_local = True
processor = WhisperProcessor.from_pretrained(model_token_pth)#"openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained(model_pth, local_files_only=is_local).to("cuda")

def map_to_pred(batch):
    audio = batch["audio"]
    input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
    batch["reference"] = processor.tokenizer._normalize(batch['sentence'])

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"))[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch

common_voice2 = common_voice1["test"].cast_column("audio", Audio(sampling_rate=16000))
result = common_voice2.map(map_to_pred)

wer = load("wer")
print(100 * wer.compute(references=result["reference"], predictions=result["prediction"]))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

88.13559322033898


In [3]:
list(zip(result["prediction"],result["reference"]))

[('fall', '否'),
 ('宋朝末年年间 定厥粉岭围', '宋朝末年年间定居粉岭围'),
 ('漸漸行動不便', '渐渐行动不便'),
 ('21年去世', '二十一年去世'),
 ('他们自称 chahara', '他们自称恰哈拉'),
 ('局部干涉的例子包括有口干 眼睛干燥以及阴道干燥', '局部干涩的例子包括有口干 眼睛干燥 及阴道干燥'),
 ('嘉靖38年 登晉世第三甲 第二名', '嘉靖三十八年 登进士第三甲第二名'),
 ('这一名称一直沿用至今', '这一名称一直沿用至今'),
 ('阿列河畔背塞', '阿列河畔贝赛'),
 ('同時喬凡尼還得到包稅合同和許多民反礦的經營權', '同时乔凡尼还得到包税合同和许多明矾矿的经营权'),
 ('為了懲罰西扎城和塞爾族的結盟 盟軍在抵達後將外城燒毀', '为了惩罚西扎城和塞尔柱的结盟 盟军在抵达后将外城烧毁'),
 ('盒內盛產黃色無魚鱗的奇色魚', '河内盛产黄色无鱼鳞的鳍射鱼'),
 ('毛丙 新木 江子为张科 新木 江子属下的一个变种', '毛柄新木姜子为樟科新木姜子属下的一个变种'),
 ('大理 尼阿拉伯 中俄聯邦又有5個個案', '达利尼亚亚扎科拉市镇是俄罗斯联邦伊尔库茨克州日加洛沃区所属的一个市镇'),
 ('主要演出泰米爾與電影', '他主要演出泰米尔语电影'),
 ('i', '为每个青少年播种梦想 点燃梦想 让更多青少年敢于有梦 勇于追梦 勤于圆梦 让每个青少年都为实现中国梦增添强大青春能量'),
 ('粗體字表示為主引', '粗体字表示为主演'),
 ('他是非洲基内亚籍佛德教独立党的成员', '他是非洲畿内亚及佛德角独立党的成员'),
 ('海洋建材在美國獨立戰爭中發揮了作用', '海洋建材在美国独立战争中发挥了作用'),
 ('福岐町市位于日本兵庫縣中部的祥政區劃', '福崎町是位于日本兵库县中部的行政区划'),
 ('下行于台 须有此所', '下行月台设有厕所'),
 ('以而合況胜以来而人 可无变化图四', '耶尔河畔圣伊莱尔人口变化图示'),
 ('進入庫 科學會議室 致力於推廣用語', '印度库鲁克文学会一直致力于推广用托隆库鲁克文书写库鲁克文学'),
 ('光緒八年 在眾舉人', '光绪八年再中举人'),
 ('台灣北部地區家庭多以在農曆年前時段包潤平和是在清明